### Ref:
- https://keras-team.github.io/keras-tuner/examples/helloworld/
- https://www.curiousily.com/posts/hackers-guide-to-hyperparameter-tuning/
- https://www.tensorflow.org/tutorials/keras/keras_tuner

In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=b3b3268df834e64d4c28687d7ae13e6501f693887a05eca263197a5e2448bb39
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=af5e6b84f0798920efe24a409b6ec2bb7b20f29f67cc0058cc8f00ac9c9987db
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from kerastuner.tuners import RandomSearch

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
"""Basic case:
- We define a `build_model` function
- It returns a compiled model
- It uses hyperparameters defined on the fly
"""

def build_model(hp):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))

  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5, 1e-6])),
                metrics=['accuracy'])
  return model

In [5]:
def hypertune_model(build_model):
  """Run hypertuning and return the optimal params"""

  # Define tuner
  tuner = RandomSearch(
      build_model,
      objective='val_accuracy',
      max_trials=5, # total number of configurations (combination of different params)
      executions_per_trial=1,
      directory='test_dir')
  
  # Show stats
  print("Search space summary....")
  tuner.search_space_summary()

  # Before running the hyperparameter search, define a callback
  # to clear the training outputs at the end of every training step.
  # import tensorflow as tf
  # import IPython

  # class ClearTrainingOutput(tf.keras.callbacks.Callback):
  #   def on_train_end(*args, **kwargs):
  #     IPython.display.clear_output(wait = True)

  # Search for best params
  tuner.search(x=x_train,
              y=y_train,
              epochs=5,
              validation_data=(x_test, y_test)) # callbacks = [ClearTrainingOutput()]
  print("Hypertune results....")
  tuner.results_summary()

  # Get the optimal hyperparameters
  best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
  print("Best learning rate is: ", best_hps.get('learning_rate'))
  return best_hps, tuner

In [6]:
# Run search
best_hps, tuner = hypertune_model(build_model)

Search space summary....


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.2341 - accuracy: 0.2557 - val_loss: 2.1139 - val_accuracy: 0.6477
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.9790 - accuracy: 0.5284 - val_loss: 1.7541 - val_accuracy: 0.7650
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.6169 - accuracy: 0.6359 - val_loss: 1.3403 - val_accuracy: 0.8081
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.2914 - accuracy: 0.6854 - val_loss: 1.0145 - val_accuracy: 0.8283
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.0570 - accuracy: 0.7192 - val_loss: 0.7963 - val_accuracy: 0.8441


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.1438 - accuracy: 0.6868 - val_loss: 0.4050 - val_accuracy: 0.8966
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4724 - accuracy: 0.8592 - val_loss: 0.2751 - val_accuracy: 0.9221
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3681 - accuracy: 0.8917 - val_loss: 0.2243 - val_accuracy: 0.9355
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3126 - accuracy: 0.9075 - val_loss: 0.1908 - val_accuracy: 0.9444
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2746 - accuracy: 0.9184 - val_loss: 0.1651 - val_accuracy: 0.9514


Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 2.3048 - accuracy: 0.1101 - val_loss: 2.3021 - val_accuracy: 0.1135
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.3024 - accuracy: 0.1086 - val_loss: 2.3021 - val_accuracy: 0.1135
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.3024 - accuracy: 0.1070 - val_loss: 2.3023 - val_accuracy: 0.1135
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.3026 - accuracy: 0.1091 - val_loss: 2.3018 - val_accuracy: 0.1135
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.3025 - accuracy: 0.1081 - val_loss: 2.3017 - val_accuracy: 0.1135


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4135 - accuracy: 0.8763 - val_loss: 0.1144 - val_accuracy: 0.9664
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1505 - accuracy: 0.9572 - val_loss: 0.0702 - val_accuracy: 0.9770
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1041 - accuracy: 0.9688 - val_loss: 0.0525 - val_accuracy: 0.9833
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0856 - accuracy: 0.9742 - val_loss: 0.0430 - val_accuracy: 0.9861
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0708 - accuracy: 0.9783 - val_loss: 0.0363 - val_accuracy: 0.9878


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1898 - accuracy: 0.9429 - val_loss: 0.0479 - val_accuracy: 0.9852
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0804 - accuracy: 0.9765 - val_loss: 0.0398 - val_accuracy: 0.9875
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0613 - accuracy: 0.9813 - val_loss: 0.0362 - val_accuracy: 0.9887
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0518 - accuracy: 0.9840 - val_loss: 0.0315 - val_accuracy: 0.9900
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0432 - accuracy: 0.9862 - val_loss: 0.0321 - val_accuracy: 0.9908


INFO:tensorflow:Oracle triggered exit
Hypertune results....


Best learning rate is:  0.001


In [7]:
#Build and compile the model with the optimal hyperparameters
model = None
model = tuner.hypermodel.build(best_hps)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [8]:
# Train best model
model.fit(x=x_train,
             y=y_train,
             epochs=5,
             validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1919 - accuracy: 0.9421 - val_loss: 0.0473 - val_accuracy: 0.9837
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0842 - accuracy: 0.9749 - val_loss: 0.0381 - val_accuracy: 0.9871
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0616 - accuracy: 0.9815 - val_loss: 0.0377 - val_accuracy: 0.9870
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0510 - accuracy: 0.9850 - val_loss: 0.0294 - val_accuracy: 0.9906
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0443 - accuracy: 0.9861 - val_loss: 0.0329 - val_accuracy: 0.9895


In [9]:
# Evaluate 
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03287776559591293
Test accuracy: 0.9894999861717224
